In [1]:
 # Common imports y set up de cada notebook

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
from tensorflow import keras
import os
import statsmodels.api as sm
from datetime import datetime
import time
import seaborn as sns

from keras.preprocessing.sequence import TimeseriesGenerator

# Para imprimir objetos grandes
#np.set_printoptions(threshold = np.inf)

# Para mantener los resultados constantes al ejecutar
np.random.seed(42) # Un random.seed constante

# Para graficar figuras bonitas
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

mpl.rcParams["figure.figsize"] = (12,6.75)
mpl.rcParams["axes.grid"] = False


# Guardar/importar imágenes

PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Guardar/importar datos
DATA_PATH = os.path.join(PROJECT_ROOT_DIR, "data")
os.makedirs(DATA_PATH, exist_ok=True)

# Guardar datasets
DATASETS = os.path.join(PROJECT_ROOT_DIR, "datasets")
os.makedirs(DATASETS, exist_ok=True)

In [2]:
df_train = pd.read_csv(os.path.join(DATA_PATH, "df_train.csv"))
df_test = pd.read_csv(os.path.join(DATA_PATH, "df_test.csv"))

formato = '%Y-%m-%d %H'

df_train['Datetime'] = pd.to_datetime(df_train["Datetime"], format=formato, utc = True)
df_train = df_train.set_index(pd.DatetimeIndex(df_train['Datetime']))
df_train.drop(["Datetime"], axis = "columns", inplace = True)

df_test['Datetime'] = pd.to_datetime(df_test["Datetime"], format=formato, utc = True)
df_test = df_test.set_index(pd.DatetimeIndex(df_test['Datetime']))
df_test.drop(["Datetime"], axis = "columns", inplace = True)

In [3]:
# Listas de características con una correlación mayor que 0.38

useful_features = ['generation_hydro_pumped_storage_consumption',
       'generation_fossil_gas', 'generation_fossil_oil',
       'generation_hydro_water_reservoir', 'price_actual', 'hour',
       'generation_fossil_hard_coal', 'generation_solar']

In [4]:
# intento crear una nueva característica sumando todas las generaciones:

generations = ['generation_biomass', 'generation_fossil_brown_coal_lignite','generation_fossil_gas',
               'generation_fossil_hard_coal', 'generation_fossil_oil',
               'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage',
               'generation_hydro_water_reservoir', 'generation_nuclear', 'generation_other',
               'generation_other_renewable', 'generation_solar', 'generation_waste', 'generation_wind_onshore']

new_features_train = pd.DataFrame(index = df_train.index)

new_features_train["Consumo_MW"] = df_train["Consumo_MW"]
new_features_train["total_generation"] = df_train[generations[0]]

for i in range(1,len(generations)):
    
    new_features_train["total_generation"] +=  df_train[generations[i]]

# Creo una nueva feture multiplicando el valor del EUR/MW por el total de MW usados

new_features_train["money_spent"] = new_features_train["total_generation"]*df_train["price_actual"]



# Añado las nuevas características a train y test, y elimino las no deseadas

In [5]:
df_train["total_generation"] = new_features_train["total_generation"]
df_train["money_spent"] = new_features_train["money_spent"]


df_train_shifted = pd.DataFrame(df_train["Consumo_MW"])
df_train_shifted[df_train.columns[1:]] = df_train[df_train.columns[1:]].shift(24)
df_train_shifted = df_train_shifted.dropna()

# Ordeno las correlaciones con valor absoluto en un dataframe

columnas = df_train_shifted.columns

correlaciones_train_df = pd.DataFrame(df_train_shifted.corr(method = "pearson").loc["Consumo_MW"])

correlaciones_ordenadas = abs(correlaciones_train_df).sort_values("Consumo_MW", ascending = False)
#correlaciones_ordenadas

#Veo que las nuevas carcterísticas son buenas para el problema deseado


#modifico el test set 

new_features_test = pd.DataFrame(index = df_test.index)

new_features_test["Consumo_MW"] = df_test["Consumo_MW"]
new_features_test["total_generation"] = df_test[generations[0]]

for i in range(1,len(generations)):
    
    new_features_test["total_generation"] +=  df_test[generations[i]]
    
new_features_test["money_spent"] = new_features_test["total_generation"]*df_test["price_actual"]

# Elimino características no deseadas

df_test["total_generation"] = new_features_test["total_generation"]
df_test["money_spent"] = new_features_test["money_spent"]

useful_features = ['total_generation', 'money_spent', 'hour',
       'generation_hydro_pumped_storage_consumption', 'price_actual',
       'generation_solar', 'generation_fossil_gas', 'x14_01d', 'x12_01n',
       'generation_hydro_water_reservoir', 'x14_01n', 'humidity_Va',
       'humidity_Ba', "Consumo_MW"]

columnas = df_train.columns

for i in range(len(columnas)): # Elimino columnas no deseadas
    
    feature = columnas[i]
    if feature not in useful_features:
        
        df_train.drop([feature], axis = 1, inplace = True)
        df_test.drop([feature], axis = 1, inplace = True)


df_train = df_train.reindex(columns = ['total_generation', 'money_spent', 'hour',
       'generation_hydro_pumped_storage_consumption', 'price_actual',
       'generation_solar', 'generation_fossil_gas', 'x14_01d', 'x12_01n',
       'generation_hydro_water_reservoir', 'x14_01n', 'humidity_Va',
       'humidity_Ba', "Consumo_MW"])
df_test = df_test.reindex(columns = ['total_generation', 'money_spent', 'hour',
       'generation_hydro_pumped_storage_consumption', 'price_actual',
       'generation_solar', 'generation_fossil_gas', 'x14_01d', 'x12_01n',
       'generation_hydro_water_reservoir', 'x14_01n', 'humidity_Va',
       'humidity_Ba', "Consumo_MW"])

In [6]:
correlaciones_ordenadas[:]

,Consumo_MW
Consumo_MW,1.000000
total_generation,0.587352
money_spent,0.539402
hour,0.402400
generation_hydro_pumped_storage_consumption,0.400341
...,...
x9_broken clouds,0.000452
rain_3h_Ma,0.000395
x8_proximity thunderstorm,0.000186
x8_overcast clouds,0.000110


In [7]:
df_train.columns.size, df_train.columns, df_test.columns

(14, Index(['total_generation', 'money_spent', 'hour',
        'generation_hydro_pumped_storage_consumption', 'price_actual',
        'generation_solar', 'generation_fossil_gas', 'x14_01d', 'x12_01n',
        'generation_hydro_water_reservoir', 'x14_01n', 'humidity_Va',
        'humidity_Ba', 'Consumo_MW'],
       dtype='object'), Index(['total_generation', 'money_spent', 'hour',
        'generation_hydro_pumped_storage_consumption', 'price_actual',
        'generation_solar', 'generation_fossil_gas', 'x14_01d', 'x12_01n',
        'generation_hydro_water_reservoir', 'x14_01n', 'humidity_Va',
        'humidity_Ba', 'Consumo_MW'],
       dtype='object'))

In [7]:
df_train

,total_generation,money_spent,hour,generation_hydro_pumped_storage_consumption,price_actual,generation_solar,generation_fossil_gas,x14_01d,x12_01n,generation_hydro_water_reservoir,x14_01n,humidity_Va,humidity_Ba,Consumo_MW
Datetime,,,,,,,,,,,,,,
2014-12-31 23:00:00+00:00,28251.0,1847897.91,23,863.0,65.41,49.0,4844.0,0.0,1.0,1899.0,1.0,77,100,25385.0
2015-01-01 00:00:00+00:00,27818.0,1805944.56,0,920.0,64.92,50.0,5196.0,0.0,1.0,1658.0,1.0,77,100,24382.0
2015-01-01 01:00:00+00:00,26796.0,1727806.08,1,1164.0,64.48,50.0,4857.0,0.0,1.0,1371.0,1.0,78,100,22734.0
2015-01-01 02:00:00+00:00,25223.0,1496228.36,2,1503.0,59.32,50.0,4314.0,0.0,1.0,779.0,1.0,78,100,21286.0
2015-01-01 03:00:00+00:00,24620.0,1379704.80,3,1826.0,56.04,42.0,4130.0,0.0,1.0,720.0,1.0,78,100,20264.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-14 13:00:00+00:00,34772.0,1590471.28,13,211.0,45.74,1083.0,4144.0,0.0,0.0,5220.0,0.0,45,55,34230.0
2018-03-14 14:00:00+00:00,35734.0,1447584.34,14,1080.0,40.51,994.0,4039.0,0.0,0.0,5360.0,0.0,45,59,33636.0
2018-03-14 15:00:00+00:00,36030.0,1252402.80,15,1272.0,34.76,716.0,4056.0,0.0,0.0,5567.0,0.0,52,59,33116.0


# Empiezo a preparar las series temporales

In [8]:
df_train.columns[-1]

'Consumo_MW'

In [9]:
#Creo dataframes de test y train para las características y para los targets


# Training set

targets = df_train.columns[-1]
features = df_train.columns[:]

features_df = df_train[features]
targets_df = pd.DataFrame(df_train[targets])

# Creo varios targets

for i in range(1,25):

    aux_df = pd.DataFrame(targets_df["Consumo_MW"].shift(-i))
    aux_str = "y + "+str(i)
    targets_df[aux_str] = aux_df["Consumo_MW"]

X_Y_df_train = features_df.copy().drop(["Consumo_MW"], axis = 1, inplace = False).join(targets_df.copy())
#X_Y_df # Contiene los dataframes de X e Y


# Test set

targets = df_test.columns[-1]
features = df_test.columns[:]

features_df = df_test[features]
targets_df = pd.DataFrame(df_test[targets])

for i in range(1,25):

    aux_df = pd.DataFrame(targets_df["Consumo_MW"].shift(-i))
    aux_str = "y + "+str(i)
    targets_df[aux_str] = aux_df["Consumo_MW"]

X_Y_df_test = features_df.copy().drop(["Consumo_MW"], axis = 1, inplace = False).join(targets_df.copy())


# Guardo los datasets

X_Y_df_train.to_csv(os.path.join(DATASETS, "df_train.csv"), index = True)
X_Y_df_test.to_csv(os.path.join(DATASETS, "df_test.csv"), index = True)